In [50]:
import json
import os
from pprint import pprint

import cohere
import numpy as np
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams

load_dotenv()


True

In [51]:
co = cohere.Client(os.getenv("COHERE_API_KEY"))
response = co.embed(["hello", "goodbye"], model='multilingual-22-12')

In [52]:
data = json.load(open('data.json'))

In [53]:
vectors = []
batch_size = 64
batch = []
print(len(data))
for row in data:
    batch.append(row['text'])
    if len(batch) >= batch_size:
        vectors.append(co.embed(batch, model='multilingual-22-12').embeddings)  # Text -> vector encoding happens here
        batch = []

if len(batch) > 0:
    vectors.append(co.embed(batch, model='multilingual-22-12').embeddings)
    batch = []
print(len(vectors), len(vectors[0]), len(vectors[1]))

vectors = np.concatenate(vectors)

242
4 64 64


In [54]:
vectors.shape

(242, 768)

In [55]:
np.save('vectors.npy', vectors, allow_pickle=False)

# Upload Model

In [56]:
qdrant_client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))
vectors_path = os.path.join(".", 'vectors.npy')
vectors = np.load(vectors_path)
vector_size = vectors.shape[1]
json_file = open('data.json', 'r')
payload = json.load(json_file)
qdrant_client.recreate_collection(
    collection_name="documents",
    vectors_config=VectorParams(size=vector_size, distance="Cosine")
)

qdrant_client.upload_collection(
    collection_name="documents",
    vectors=vectors,
    payload=payload,
    ids=None,
    batch_size=256,
    parallel=2
)

# Test Query
The answer is
  - $ 3 + 5 = 8$
  - $ 3 + 4 = 7$


In [57]:
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
# Take a random description as a query
sample_query = "lower costs"
print(sample_query)

lower costs


In [59]:
query_vector = co.embed([sample_query], model='multilingual-22-12').embeddings[
    0]  # Convert query description into a vector.

In [60]:
# scores = cosine_similarity([query_vector], vectors)[0]  # Look for the most similar vectors, manually score all vectors
# top_scores_ids = np.argsort(scores)[-5:][::-1]  # Select top-5 with vectors the largest scores
print(os.getenv("QDRANT_URL"), os.getenv("QDRANT_API_KEY"))
qdrant_client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))
print("client loaded")
query_vector = np.array(query_vector).astype(np.float64)
print("query_vector")
hits = qdrant_client.search(
    collection_name="documents",
    query_vector=query_vector,
    limit=5
)

https://2e366342-0e12-4f7e-8772-cc72f8f39464.us-east-1-0.aws.cloud.qdrant.io:6333 nbpdkQEEHRK5s9K2WzznUqU5ZKyv2bCAEl9mAlleangh5UcQzsXFzA
client loaded
query_vector


In [61]:
pprint([hit.payload for hit in hits])


[{'meta': {'classification': 'memo',
           'finished': 'finished',
           'source': 'quarterly.md'},
  'text': '## Response:\n'
          'In an effort to cut costs we’ve made the decision to reduce '
          'headcount.'},
 {'meta': {'classification': 'proposal',
           'finished': 'finished',
           'source': 'ai_proposal.md'},
  'text': 'Thank you for considering this proposal.'},
 {'meta': {'classification': 'guide',
           'finished': 'finished',
           'source': 'website_guide.md'},
  'text': "This can improve your website's"},
 {'meta': {'classification': 'proposal',
           'finished': 'finished',
           'source': 'dummy_proposal.md'},
  'text': 'Increase revenue and market share through innovative technology '
          'solutions.'},
 {'meta': {'classification': 'guide',
           'finished': 'finished',
           'source': 'AWSGuide.md'},
  'text': '* Use cost optimization tools: Cloud providers offer cost '
          'optimization tools t